<a href="https://colab.research.google.com/github/Dominika26/machine-learning-bootcamp/blob/main/Naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Naive Bayes**- radzi sobie dobrze z dużą ilością danych 


*   P(y|X)=(P(X|y)*P(y))/P(X)
*   Zdarzenia A i B są niezależne, gdy P(A)*P(B)
* Dlaczego naiwny- bo zakaładamy, że cechy w naszym modelu są od siebie niezależne, pozwala to uprościć obliczenia i nie wpływa to aż tak na działanie modelu. Działa dobrze w wielu rzeczywistych sytuacjach 
* y jest zmienną docelową 
* X jest wektorem zmiennych objaśniających (x1, x2, x3, x4, ..., xn) 
*  P(y|x1, x2, x3, x4)=(P(x1, x2, x3, x4|y)*P(y))/P(x1, x2, x3, x4
)
* Ale dzięki założeniu o niezależności mogę rozdzielić P(x1, x2, x3, x4|y) * P(y)
na P(x1|y)* P(x2|y) * P(x3|y) * P(x4|y) * P(y)



Naive bayes w pryzpadku danych ciągłych:


*   przyjmujemy założenie o rozkładzie normalnym 



In [1]:
!pip install scikit-learn

In [2]:
!pip install --upgrade scikit-learn 

In [4]:
import numpy as np
import pandas as pd 
import sklearn 

np.random.seed(42)
np.set_printoptions(precision=6, suppress=True)
sklearn.__version__

'1.0.2'

In [5]:
pogoda = ['słonecznie', 'deszczowo', 'pochmurno', 'deszczowo', 'słonecznie', 'słonecznie',
          'pochmurno', 'pochmurno', 'słonecznie']
temperatura = ['ciepło', 'zimno', 'ciepło', 'ciepło', 'ciepło', 'umiarkowanie',
               'umiarkowanie', 'ciepło', 'zimno']

spacer = ['tak', 'nie', 'tak', 'nie', 'tak', 'tak', 'nie', 'tak', 'nie']      

raw_df = pd.DataFrame(data={'pogoda': pogoda, 'temperatura': temperatura, 'spacer': spacer})
df = raw_df.copy()
df

,pogoda,temperatura,spacer
0,słonecznie,ciepło,tak
1,deszczowo,zimno,nie
2,pochmurno,ciepło,tak
3,deszczowo,ciepło,nie
4,słonecznie,ciepło,tak
5,słonecznie,umiarkowanie,tak
6,pochmurno,umiarkowanie,nie
7,pochmurno,ciepło,tak
8,słonecznie,zimno,nie


Przygotowanie danych do modelu 

In [6]:
# mapowanie zmiennej docelowej- tak na 1, nie na 0
from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder()
df['spacer'] = encoder.fit_transform(spacer)
df

,pogoda,temperatura,spacer
0,słonecznie,ciepło,1
1,deszczowo,zimno,0
2,pochmurno,ciepło,1
3,deszczowo,ciepło,0
4,słonecznie,ciepło,1
5,słonecznie,umiarkowanie,1
6,pochmurno,umiarkowanie,0
7,pochmurno,ciepło,1
8,słonecznie,zimno,0


In [7]:
# mapowanie zmiennych objaśniających- czyli jakby rodziela mi te wartości na więcej cech i dla każdego robi tal/nie, 0/1
df = pd.get_dummies(df, columns=['pogoda', 'temperatura'], drop_first=True)
df


,spacer,pogoda_pochmurno,pogoda_słonecznie,temperatura_umiarkowanie,temperatura_zimno
0,1,0,1,0,0
1,0,0,0,0,1
2,1,1,0,0,0
3,0,0,0,0,0
4,1,0,1,0,0
5,1,0,1,1,0
6,0,1,0,1,0
7,1,1,0,0,0
8,0,0,1,0,1


In [8]:
# usuwam z nowego zbioru zmienną docelową
data = df.copy()
target = data.pop('spacer')
data

,pogoda_pochmurno,pogoda_słonecznie,temperatura_umiarkowanie,temperatura_zimno
0,0,1,0,0
1,0,0,0,1
2,1,0,0,0
3,0,0,0,0
4,0,1,0,0
5,0,1,1,0
6,1,0,1,0
7,1,0,0,0
8,0,1,0,1


In [9]:
target

0    1
1    0
2    1
3    0
4    1
5    1
6    0
7    1
8    0
Name: spacer, dtype: int64

**Klasyfikator bayesowski**

In [10]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(data, target)

GaussianNB()

In [11]:
model.score(data, target)
# dokładność naszego modelu wynosi 77,78%

0.7777777777777778

In [12]:
# chcemy dokonać predykcji na pierwszym wierszu 
data.iloc[[0]]

,pogoda_pochmurno,pogoda_słonecznie,temperatura_umiarkowanie,temperatura_zimno
0,0,1,0,0


In [13]:
model.predict(data.iloc[[0]])

array([1])

In [14]:
# tu mogę sprawdzić że rzeczywiście przy słonecznej i ciepłej pogodzie należy wyjść na spacer
encoder.classes_

array(['nie', 'tak'], dtype='<U3')

In [15]:
encoder.classes_[model.predict(data.iloc[[0]])[0]]

'tak'

In [16]:
# jeżeli chcemy przewidzieć dokładne prawdopodobieństwo. Prawdopodobieństwo 'tak' jest 
# bardzo wysokie 
model.predict_proba(data.iloc[[0]])

array([[0.000006, 0.999994]])